<a href="https://colab.research.google.com/github/lrzcalmon/Repo_Git_Api/blob/main/Tech_Challenge_03_Llama_3_1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carrega as bibliotecas

Montar com permissão de leitura e escrita o Google Drive no ambiente do Google Colab. Isto evita a carga de arquivos e perda de resultados gravados.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importação das bibliotecas necessárias para o processamento de dados.

In [2]:
import pandas as pd
import unicodedata
import json
import re

import os

# Desabilitar para não solicitar APIKey
os.environ["WANDB_DISABLED"] = "true"


###Prepapação do Dataset

In [4]:
# Efetua as diversas limpezas e estabilizações do dataset
def prepara_dataset(tam_regs=None, inpfile, outfile):
# Primeiro carrega o JSON como uma tupla no vetor data
    data = []
    with open(inpfile, 'r', encoding='utf-8') as file:
        for line in file: # para cada linha
            try:
                item = json.loads(line)
                if 'title' in item and 'content' in item:
                    title = item['title']
                    content = item['content']
                    if title or content is not None:
                        data.append({'title': title, 'content': content})  # só nos interessa tituloe conteudo
            except json.JSONDecodeError:
                pass # bypass de leituras erradas

    df = pd.DataFrame(data) # Usa o pandas para converter uma lista em dataFrame

    # Função para remover carac de controle e excesso de espaço para ganhar tokens
    def normalizar(text):
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalizar)
    df['content'] = df['content'].apply(normalizar)

    df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')] # remove brancos e vazios
    df.drop_duplicates(subset=['title', 'content'], inplace=True) # remove duplicados

    df.reset_index(drop=True, inplace=True) # como mexemos, precisa resetar o índice

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Responda à pergunta com base no título do produto."
    df['input'] = df['title'].apply(lambda x: f"O que é '{x}'?")
    df['output'] = df['content']

    # Se tam_regs definir valor de corte
    if tam_regs is not None:
        df = df.sample(n=tam_regs, random_state=42).reset_index(drop=True)

    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records') # Formata para dicionários

    with open(outfile, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4) # Salva o Json para uso

    print(f"Total de registros processados: {len(df)}")

In [5]:
dataset_treino = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trn.json"
dataset_treino_output = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_train_dataset.json"

dataset_teste = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/tst.json"
dataset_teste_output = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_test_dataset.json"

Processa os datasets.

In [6]:
process_and_format_dataset(tam_regs=250000, dataset_treino, dataset_treino_output)  # 250.000 registros para treinamento
process_and_format_dataset(tam_regs=2500, teste_dataset, dataset_teste_output)     # 2.500 registros para teste

Dataset processado e salvo em '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_train_dataset.json'. Total de registros: 250000
Dataset processado e salvo em '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_test_dataset.json'. Total de registros: 2500


###Instalação do Unsloth e bibliotecas
Instalação do Unsloth em uma instância **gratuita** do Google Colab, a **Tesla T4**, para o fine-tuning com o modelo opensource **Llama 3.1-8B**.

In [7]:
!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git" unsloth-zoo
!pip install peft accelerate bitsandbytes triton --no-deps xformers "trl<0.9.0" dataset

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-3i2ooyrd/unsloth_3d485ff4e58f4a5e9e59efba17613445
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-3i2ooyrd/unsloth_3d485ff4e58f4a5e9e59efba17613445
  Resolved https://github.com/unslothai/unsloth.git to commit 6e5534dc860005be7fd7790b3aa73c2a3996aa5e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


###Configuração do modelo


Lista de modelos da unsloth quantizados para 4bits para reduzir o consumo de memória e permitir o treinamento do modelo em GPUs com menos VRAM ou gratuitas como a **Tesla T4**.

In [9]:
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",           # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",         # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",      # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",             # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",                # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",                 # Gemma 2x faster!
]                                                   # More models at https://huggingface.co/unsloth

Definição da quantidade de tokens, tipo de dados, quantização e modelo.

In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 128                                  # Define o comprimento máximo das sequências de entrada.
dtype = None                                          # Tipo de dados (também pode ser torch.float16 ou torch.float32).
load_in_4bit = True                                   # Indica que o modelo será quantizado em 4 bits para economizar memória.
model_name = "unsloth/Meta-Llama-3.1-8B"              # Modelo escolhido para executar o fine-tuning

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


###PEFT e LoRA

Configuração do modelo para o treinamento eficiente usando LoRA onde
apenas as camadas lineares do modelo serão treinadas, adaptando o modelo para tarefas específicas mantendo a maioria do conhecimento pré-treinado intacto e reduzindo a quantidade de dados necessário para efetuar o ajuste.


In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                                                     # Rank das matrizes LoRA podendo ser 8, 16, 32, 64, 128. Quanto menor o valor, menos custo computacional e menos poder de ajuste.

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",   # Módulos do modelo que serão adaptados
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,                                            # Controla a escala de atualização das matrizes LoRA
    lora_dropout = 0,                                           # Ajuda a prevenir overfitting em tarefas específicas, onde 0 é otimizado e valores como 0.1 ou 0.2 podem ser úteis para baixo volume de dados.
    bias = "none",                                              # Nenhuma definição necessária para manter a otimização

    use_gradient_checkpointing = "unsloth",                     # Utilização de 30% menos de VRAM e 2x mais rápido para grandes contextos
    random_state = 3407,                                        # Número randômico

    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


###Prompt

Definição do formato do prompt seguintdo um template consistente para ajudar o modelo a entender melhor a estrutura da tarefa.



In [12]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


Aplicação do template de prompt a cada registro do dataset retornando um dicionário com a chave 'text' contendo os prompts formatados.

In [13]:
EOS_TOKEN = tokenizer.eos_token            # Indicação de fim de linha para evitar repetições
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


##Inferência

Carrega e prepara o modelo para inferência com as otimizações realizadas.

In [14]:
FastLanguageModel.for_inference(model) # Habilita inferência nativa que é 2x mais rápida


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

###Teste de prompt antes do fine-tuning com o dataset formatado


Prompt de teste antes de realizer o fine-tuning do modelo que não encontrará a resposta correta para a pergunta do usuário.

In [15]:
inputs = tokenizer(
[
    alpaca_prompt.format(
      "Answer the question based on the product title.",  # instruction
      # "What is 'Girls Ballet Tutu Neon Pink'?",           # input
      # "Who was Ayrton Senna?",                            # input
      "Last Pink FLoyd Album?",                           # input
      "",                                                 # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product title.

### Input:
Last Pink FLoyd Album?

### Response:
Response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to


###Dataset


Mapeamento do dataset formatado e gerado anteriormente pela preparação de dados.

In [16]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_train_dataset.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/250000 [00:00<?, ? examples/s]

### Treinamento do modelo

Definição de parâmetros do treinamento do modelo utilizando o `SFTTrainer` com `60 steps` para melhorar o desempenho, mas também pode ser utilizado `num_train_epochs=1` para uma execução completa, e desabilitado `max_steps=None`.

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

output_dir = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M"

args = TrainingArguments(

    per_device_train_batch_size = 1,        # Tamanho do batch por GPU reduzido para 1 para evitar falta de memória
    gradient_accumulation_steps = 2,        # Número de passos de acumulação de gradientes antes de atualizar os pesos do modelo
    warmup_steps = 5,                       # Número de passos iniciais de treinamento nos quais a taxa de aprendizado é gradualmente aumentada.

    # num_train_epochs = 1,                 # Número de épocas de treinamento, desabilitado neste caso, pois o número de passos é limitado pelo max_steps
    max_steps = 60,                         # Número máximo de passos de treinamento a serem executados.
    learning_rate = 2e-4,                   # Taxa de aprendizado inicial do otimizador.

    fp16 = not is_bfloat16_supported(),     # Meio-precisão (16 bits) para econima de memória.
    bf16 = is_bfloat16_supported(),         # Formato bfloat16 (mais estável em algumas arquiteturas de hardware).

    logging_steps = 1,                      # Frequência (em passos) com que as métricas de log são registradas.
    optim = "adamw_8bit",                   # versão de Adam otimizada para economizar memória usando pesos de 8 bits.
    weight_decay = 0.01,                    # Penalidade aplicada aos pesos do modelo para evitar overfitting.
    lr_scheduler_type = "linear",           # Reduz a taxa de aprendizado de forma linear ao longo do treinamento.
    seed = 3407,                            # Controle de aleatoriedade, garantindo a reprodutibilidade.
    output_dir = output_dir,                # Diretório onde os resultados do treinamento (e possivelmente checkpoints) serão armazenados.
    report_to = "none",                     # Relatório e monitoramento de treinamento

    #evaluation_strategy="no",               # Sem avaliação durante o treinamento
    save_strategy="no",                     # Não salva checkpoints intermediários

)

trainer = SFTTrainer(

    model = model,                          # Modelo escolhido para executar o fine-tuning
    tokenizer = tokenizer,                  # Tokenizador do modelo que transforma texto em tensores numéricos
    train_dataset = dataset,                # Conjunto de dados de treinamento formatado
    dataset_text_field = "text",            # Nome do campo no dataset que contém o texto que será usado no treinamento
    max_seq_length = max_seq_length,        # Comprimento máximo da sequência de tokens
    dataset_num_proc = 2,                   # Número de processos para carregar e processar os dados do dataset
    packing = False,                        # Pode fazer o treinamento 5x mais rápido com pequenas sequências

    args = args,                            # Argumentos para o treinamento
)

trainer.train()

Map (num_proc=2):   0%|          | 0/250000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 250,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,0.274300
2,0.377000
3,0.369800
4,0.487200
5,0.181600
6,0.130200
7,0.073100
8,0.095200
9,0.071600
10,0.053900


TrainOutput(global_step=60, training_loss=0.04051717385752151, metrics={'train_runtime': 98.2267, 'train_samples_per_second': 1.222, 'train_steps_per_second': 0.611, 'total_flos': 629363485974528.0, 'train_loss': 0.04051717385752151, 'epoch': 0.00048})

###Exportação do modelo

Exportação do modelo e o tokenizador treinados para uso futuro sem a necessidade de executar o fine-tuning novamente.

In [18]:
model.save_pretrained("/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model")

('/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model/tokenizer.json')

###Utilização do modelo com fine-tuning

Carrega o treinado com o fine-tuning e prepara o modelo para inferência com as otimizações realizadas.

In [19]:
!pip install unsloth
from unsloth import FastLanguageModel

model_name = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model"

max_seq_length = 128                                  # Define o comprimento máximo das sequências de entrada.
dtype = None                                          # Tipo de dados (também pode ser torch.float16 ou torch.float32).
load_in_4bit = True                                   # Indica que o modelo será quantizado em 4 bits para economizar memória.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

Prompt depois de realizer o fine-tuning do modelo que encontrará a resposta correta para a pergunta do usuário.

In [26]:
inputs = tokenizer(
[
    alpaca_prompt.format(
      "Answer the question based on the product content.",  # instruction
      #"What is 'Girls Ballet Tutu Neon Pink'?",           # input
      #"Who was Ayrton Senna?",                            # input
      "Last Album?",                           # input

      "",                                                 # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=500)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product content.

### Input:
Last Album?

### Response:
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###